# Tutorial configuration

In [2]:
!rm -rf snapearthapis
!git clone https://github.com/chicham/snapearthapis.git
!pip install -r snapearthapis/requirements.txt
!!add-apt-repository ppa:longsleep/golang-backports -y
!apt-get update > /dev/null && apt-get -y install jq golang-go > /dev/null
!go version
!!go install github.com/fullstorydev/grpcurl/cmd/grpcurl@latest
import os
os.environ['PATH'] += ":/root/go/bin"

Cloning into 'snapearthapis'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 163 (delta 62), reused 144 (delta 43), pack-reused 0
Receiving objects: 100% (163/163), 39.31 KiB | 258.00 KiB/s, done.
Resolving deltas: 100% (62/62), done.
go version go1.18.1 linux/amd64


# EarthSignature Tutorial

## Introduction

The goal of this notebook is to show how to use the EarthSignature API to download segmented Sentinel2 products. At the end of this tutorial, you will be able to query the API, understand how to filter the results when querying the API and understand what are the data returned by the API.


## Visualize the products available

In [6]:
!pip install orjson
import orjson as json 
lines = !grpcurl -insecure  -max-msg-sz 1073741824 earthsignature.snapearth.eu:443 snapearth.api.v1.database.DatabaseProductService.ListProductIds | jq -c -M
products = [json.loads(line) for line in lines]

     |████████████████████████████████| 253 kB 4.2 MB/s 


[{'creationDate': '2022-04-30T00:00:00Z',
  'productId': 'S2B_MSIL1C_20220430T100019_N0400_R122_T33TXL_20220430T120209',
  'publicationDate': '2022-04-30T00:00:00Z',
  'wkt': 'MULTIPOLYGON (((16.2829359999999994 45.6240570000000005, 16.3153690000000005 45.7014489999999967, 16.3161799999999992 45.7033819999999977, 16.3759319999999988 45.8458579999999998, 16.4038110000000010 45.9120470000000012, 16.4374579999999995 45.9921140000000008, 16.4590020000000017 46.0433489999999992, 16.2925320000000013 46.0462649999999982, 16.2829359999999994 45.6240570000000005)))'}]

In [11]:
import base64

import folium
from shapely import wkt
from shapely.geometry import mapping


EUROPE_COORDINATES = wkt.loads(
    "POLYGON((-10.61 71.16, 44.85 71.16, 44.85 35.97, -10.61 35.97, -10.61 71.16))",
)
MAP_CENTER = mapping(EUROPE_COORDINATES.centroid)

In [32]:

map_ = folium.Map(
    location=MAP_CENTER["coordinates"][::-1],
    zoom_start=4,
    crs="EPSG3857",
)


for product in products:
    geom = wkt.loads(product['wkt'])
    folium.GeoJson(data=geom).add_to(map_)
    centroid = mapping(geom.centroid)
    folium.Marker(
            location=centroid["coordinates"][::-1],
            tooltip=f'product["productId"]',
        ).add_to(map_)    

map_


ValueError: ignored

In [33]:
folium.__version__

'0.2.1'

## Query the API

drive.mount("/content/drive")

with grpc.insecure_channel(f"{host}", options) as channel:
    stub = database_pb2_grpc.DatabaseProductServiceStub(channel)
    request = database_pb2.ListSegmentationRequest()
    responses = stub.ListSegmentationRequest(request)
    # Write one directory per product
    # Write cloud_mask and segmentation